This a rnn with keras used for doing  sentiment analysis on tweets
There is a dataset of 16000 lines and the task is to describe  each tweet as positive or negative.
Although data is here and will also be uploaded with the code.
Here's the link to download the dataset :
https://github.com/crwong/cs224u-project/tree/master/data/sentiment

In [0]:
#importing the required libraries

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.convolutional import Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np


In [6]:
!pip3 install spacy
!python -m spacy download en

    100% |████████████████████████████████| 17.6MB 2.3MB/s 
    100% |████████████████████████████████| 92kB 19.7MB/s 
    100% |████████████████████████████████| 1.2MB 19.3MB/s 
    100% |████████████████████████████████| 51kB 16.8MB/s 
    100% |████████████████████████████████| 194kB 18.5MB/s 
    100% |████████████████████████████████| 71kB 20.9MB/s 
    100% |████████████████████████████████| 604kB 21.8MB/s 
    100% |████████████████████████████████| 51kB 19.9MB/s 
    100% |████████████████████████████████| 389kB 9.6MB/s 
    100% |████████████████████████████████| 143kB 22.1MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | /

 - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/fb/00/28/75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for murmurhash ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for cymem ... - \ done
  Stored in directory: /content/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13
  Running setup.py bdist_wheel for preshed ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/8f/85/06/2d132fb649a6bbcab22487e4147880a55b0dd0f4b18fdfd6b5
  Running setup.py bdist_wheel for thinc ... - \ |

 - \ | / done
  Stored in directory: /content/.cache/pip/wheels/f8/b1/86/c92e4d36b690208fff8471711b85eaa6bc6d19860a86199a09
  Running setup.py bdist_wheel for msgpack-python ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
Successfully built spacy murmurhash cymem preshed thinc pathlib ujson dill regex wrapt cytoolz msgpack-python
    89% |████████████████████████████▊   | 33.5MB 21.6MB/s eta 0:00:01

    100% |████████████████████████████████| 37.4MB 46.6MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
# read after completion
import spacy

nlp=spacy.load("en")


In [24]:
#load the dataset

#uploading the dataset

from google.colab import files
uploaded = files.upload()


#!wget -c https://raw.githubusercontent.com/crwong/cs224u-project/master/data/sentiment/training.1600000.processed.noemoticon.csv -O twitter.csv

--2018-06-06 04:29:36--  https://raw.githubusercontent.com/crwong/cs224u-project/master/data/sentiment/training.1600000.processed.noemoticon.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [9]:
#import the dataset

train = pd.read_csv("twitter.csv", encoding = "latin - 1")

y = train[train.columns[0]]
X = train[train.columns[5]]

'''print(X)

print(y)
'''


'print(X)\n\nprint(y)\n'

In [0]:
# split the data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size = 0.2, random_state = 42)
y_test = pd.get_dummies(y_test) #one hot encoding ke similar

In [0]:
#function to remove stopwords

def stopwords(sent):
  new = []
  sent = nlp(sent)
  for w in sent:
    if(w.is_stop == False) & (w.pos_ != "PUNCT"):
      new.append(w.string.strip())
    c = " ".join(str(x) for x in new)
  return c

In [0]:
# function to lemmatize the tweets
def lemma(sent):
  sent = nlp(sent)
  str = ""
  for w in sent:
    str += " " + w.lemma_
  return nlp(str)
  

In [0]:
#loading the glove model (basically a step in vectorizing the corpus)

def loadGloveModel(gloveFile):
  print("Loading Glove Model")
  
  f = open(gloveFile, 'r')
  model = {}
  for line in f:
    splitLine = line.split()
    word = splitLine[0]
    embedding = [float(val) for val in splitLine[1:]]
    model[word] = embedding
  print("Done."),len(model), (" words loaded!")
  return model

I have downloaded the glove file on the virtual machine instead of doing it on the local machine. And later extracted that using zipfile

In [14]:

!wget "http://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip"

--2018-06-06 04:22:47--  http://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip [following]
--2018-06-06 04:22:48--  https://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408741 (1.4G) [application/zip]
Saving to: ‘glove.twitter.27B.zip’

glove.twitter.27B.z  29%[====>               ] 427.98M  14.0MB/s    eta 81s    

glove.twitter.27B.z 100%[===================>]   1.42G  19.3MB/s    in 98s     

2018-06-06 04:24:26 (14.8 MB/s) - ‘glove.twitter.27B.zip’ saved [1520408741/1520408741]



In [15]:
from zipfile import ZipFile

file_name = "glove.twitter.27B.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Done")
  

Done


In [16]:
# save the glove model

model = loadGloveModel("glove.twitter.27B.200d.txt")

Loading Glove Model
Done.


In [0]:
#vectorizing the sentences
def sent_vectorizer(sent, model):
  
  sent_vec = np.zeros(200)
  numw = 0
  for w in sent.split():
    try:
      
      sent_vec = np.add(sent_vec, model[str(w)])
      numw += 1
    except:
      pass
    return sent_vec
  
  

In [0]:
#obtain a clean vector

cleanvector = []
for i in range(X_test.shape[0]):
  document = X_test[i]
  document = document.lower()
  document = lemma(document)
  document = str(document)
  cleanvector.append(sent_vectorizer(document, model))

In [0]:
#Getting the input and output in proper shape

cleanvector = np.array(cleanvector)
cleanvector = cleanvector.reshape(len(cleanvector), 200, 1)

In [20]:
#tokenizing the sequences

tokenizer = Tokenizer(num_words=16000)
tokenizer.fit_on_texts(X_test)
sequences = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index
print('Found %s unique tokens' %len(word_index))
data = pad_sequences(sequences, maxlen = 15, padding = 'post')
print(data.shape)

Found 9850 unique tokens
(4000, 15)


In [0]:
#reshape the data and prepair to train
data = data.reshape(len(cleanvector), 15, 1)
from sklearn.model_selection import train_test_split
trainx, validx, trainy, validy = train_test_split(data, y_test, test_size = 0.3, random_state = 42)

In [22]:
#calculate the number of words
nb_words = len(tokenizer.word_index) + 1
print(nb_words)

9851


In [23]:
#obtain the embedding matrix

embedding_matrix = np.zeros((nb_words, 200))
for word, i in word_index.items():
  embedding_vector = model.get(word) #word ki vector representation mil rahi h
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis = 1) == 0))
    

Null word embeddings: 2801


In [0]:
#converting the raininf data to numpy array

trainy = np.array(trainy)
validy = np.array(validy)


In [0]:
#building a simple rnn network
def modelbuild():
  model = Sequential()
  model.add(keras.layers.InputLayer(input_shape=(15,1)))
  keras.layers.embeddings.Embedding(nb_words, 15, weights = [embedding_matrix], input_length= 15, trainable = False)
  
  model.add(keras.layers.recurrent.SimpleRNN(units = 500, activation = 'relu', use_bias = True))
  model.add(keras.layers.Dense(units = 1000, input_dim = 2000, activation = 'sigmoid'))
  model.add(keras.layers.Dense(units = 500, input_dim = 1000, activation = 'relu'))
  model.add(keras.layers.Dense(units = 2, input_dim = 500, activation = 'softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  
  return model

In [43]:
#compiling the model
final_model = modelbuild()
final_model.fit(trainx, trainy, epochs = 200, batch_size = 120, validation_data = (validx, validy))

Train on 2800 samples, validate on 1200 samples
Epoch 1/200
2800/2800 [==============================] - 1s 317us/step - loss: 0.9008 - acc: 0.5150 - val_loss: 0.7227 - val_acc: 0.4817
Epoch 2/200
2800/2800 [==============================] - 0s 164us/step - loss: 0.7118 - acc: 0.5018 - val_loss: 0.6925 - val_acc: 0.5200
Epoch 3/200
2800/2800 [==============================] - 0s 161us/step - loss: 0.7302 - acc: 0.5064 - val_loss: 0.7013 - val_acc: 0.5183
Epoch 4/200
2800/2800 [==============================] - 0s 175us/step - loss: 0.7206 - acc: 0.5079 - val_loss: 0.6933 - val_acc: 0.5183
Epoch 5/200
2800/2800 [==============================] - 0s 169us/step - loss: 0.7094 - acc: 0.5004 - val_loss: 0.6992 - val_acc: 0.5183
Epoch 6/200
2800/2800 [==============================] - 0s 170us/step - loss: 0.7017 - acc: 0.4929 - val_loss: 0.7009 - val_acc: 0.5183
Epoch 7/200
2800/2800 [==============================] - 0s 174us/step - loss: 0.6966 - acc: 0.5111 - val_loss: 0.7323 - val_acc: 

2800/2800 [==============================] - 0s 169us/step - loss: 0.7071 - acc: 0.5086 - val_loss: 0.6928 - val_acc: 0.5183
Epoch 13/200
2800/2800 [==============================] - 0s 169us/step - loss: 0.6986 - acc: 0.4943 - val_loss: 0.6925 - val_acc: 0.5183
Epoch 14/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.7015 - acc: 0.5007 - val_loss: 0.7000 - val_acc: 0.4817
Epoch 15/200
2800/2800 [==============================] - 0s 167us/step - loss: 0.6980 - acc: 0.5029 - val_loss: 0.6925 - val_acc: 0.5183
Epoch 16/200
2800/2800 [==============================] - 0s 167us/step - loss: 0.6995 - acc: 0.5114 - val_loss: 0.7000 - val_acc: 0.4817
Epoch 17/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.7032 - acc: 0.4936 - val_loss: 0.6925 - val_acc: 0.5183
Epoch 18/200
2800/2800 [==============================] - 0s 173us/step - loss: 0.6976 - acc: 0.4950 - val_loss: 0.6953 - val_acc: 0.4817
Epoch 19/200
2800/2800 [=======================

Epoch 24/200
2800/2800 [==============================] - 0s 172us/step - loss: 0.6970 - acc: 0.5029 - val_loss: 0.6931 - val_acc: 0.5183
Epoch 25/200
2800/2800 [==============================] - 0s 172us/step - loss: 0.6955 - acc: 0.4893 - val_loss: 0.6936 - val_acc: 0.4817
Epoch 26/200
2800/2800 [==============================] - 0s 172us/step - loss: 0.7080 - acc: 0.5014 - val_loss: 0.6934 - val_acc: 0.5183
Epoch 27/200
2800/2800 [==============================] - 0s 174us/step - loss: 0.7087 - acc: 0.5093 - val_loss: 0.6925 - val_acc: 0.5183
Epoch 28/200
2800/2800 [==============================] - 0s 177us/step - loss: 0.6945 - acc: 0.5071 - val_loss: 0.6975 - val_acc: 0.5183
Epoch 29/200
2800/2800 [==============================] - 0s 177us/step - loss: 0.6974 - acc: 0.4911 - val_loss: 0.7015 - val_acc: 0.4817
Epoch 30/200
2800/2800 [==============================] - 0s 176us/step - loss: 0.6964 - acc: 0.5050 - val_loss: 0.6925 - val_acc: 0.5183
Epoch 31/200
2800/2800 [==========

2800/2800 [==============================] - 0s 171us/step - loss: 0.6945 - acc: 0.4943 - val_loss: 0.6985 - val_acc: 0.4817
Epoch 36/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6941 - acc: 0.5129 - val_loss: 0.7019 - val_acc: 0.4817
Epoch 37/200
2800/2800 [==============================] - 0s 169us/step - loss: 0.6939 - acc: 0.5036 - val_loss: 0.6926 - val_acc: 0.5183
Epoch 38/200
2800/2800 [==============================] - 0s 170us/step - loss: 0.6949 - acc: 0.5100 - val_loss: 0.6925 - val_acc: 0.5183
Epoch 39/200
2800/2800 [==============================] - 0s 169us/step - loss: 0.6946 - acc: 0.5121 - val_loss: 0.6943 - val_acc: 0.5183
Epoch 40/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6943 - acc: 0.5000 - val_loss: 0.6926 - val_acc: 0.5183
Epoch 41/200
2800/2800 [==============================] - 0s 170us/step - loss: 0.6946 - acc: 0.4971 - val_loss: 0.6966 - val_acc: 0.4817
Epoch 42/200
2800/2800 [=======================

2800/2800 [==============================] - 0s 171us/step - loss: 0.6944 - acc: 0.5000 - val_loss: 0.6973 - val_acc: 0.4817
Epoch 47/200
2800/2800 [==============================] - 0s 173us/step - loss: 0.6967 - acc: 0.4829 - val_loss: 0.6925 - val_acc: 0.5183
Epoch 48/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6952 - acc: 0.5143 - val_loss: 0.6938 - val_acc: 0.5183
Epoch 49/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6947 - acc: 0.4993 - val_loss: 0.6926 - val_acc: 0.5183
Epoch 50/200
2800/2800 [==============================] - 0s 173us/step - loss: 0.6953 - acc: 0.4914 - val_loss: 0.6925 - val_acc: 0.5183
Epoch 51/200
2800/2800 [==============================] - 0s 172us/step - loss: 0.6952 - acc: 0.4836 - val_loss: 0.6926 - val_acc: 0.5183
Epoch 52/200
2800/2800 [==============================] - 0s 174us/step - loss: 0.6952 - acc: 0.4982 - val_loss: 0.6944 - val_acc: 0.4817
Epoch 53/200
2800/2800 [=======================

...

Epoch 151/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6932 - acc: 0.4986 - val_loss: 0.6931 - val_acc: 0.5183
Epoch 152/200
2800/2800 [==============================] - 0s 175us/step - loss: 0.6932 - acc: 0.4957 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 153/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6932 - acc: 0.4957 - val_loss: 0.6931 - val_acc: 0.5183
Epoch 154/200
2800/2800 [==============================] - 0s 173us/step - loss: 0.6931 - acc: 0.5036 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 155/200
2800/2800 [==============================] - 0s 173us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 156/200
2800/2800 [==============================] - 0s 174us/step - loss: 0.6932 - acc: 0.4907 - val_loss: 0.6931 - val_acc: 0.5183
Epoch 157/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6932 - acc: 0.4957 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 158/200
2800/2800 [==

2800/2800 [==============================] - 0s 177us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6933 - val_acc: 0.4817
Epoch 164/200
2800/2800 [==============================] - 1s 181us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6935 - val_acc: 0.4817
Epoch 165/200
2800/2800 [==============================] - 0s 169us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6934 - val_acc: 0.4817
Epoch 166/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6933 - val_acc: 0.4817
Epoch 167/200
2800/2800 [==============================] - 0s 172us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6934 - val_acc: 0.4817
Epoch 168/200
2800/2800 [==============================] - 0s 174us/step - loss: 0.6931 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 169/200
2800/2800 [==============================] - 1s 180us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 170/200
2800/2800 [================

2800/2800 [==============================] - 0s 166us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 176/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6933 - val_acc: 0.4817
Epoch 177/200
2800/2800 [==============================] - 0s 172us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 178/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6933 - val_acc: 0.4817
Epoch 179/200
2800/2800 [==============================] - 0s 172us/step - loss: 0.6931 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 180/200
2800/2800 [==============================] - 0s 167us/step - loss: 0.6932 - acc: 0.4679 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 181/200
2800/2800 [==============================] - 0s 172us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 182/200
2800/2800 [================

Epoch 187/200
2800/2800 [==============================] - 0s 170us/step - loss: 0.6932 - acc: 0.4850 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 188/200
2800/2800 [==============================] - 0s 170us/step - loss: 0.6932 - acc: 0.4914 - val_loss: 0.6931 - val_acc: 0.5183
Epoch 189/200
2800/2800 [==============================] - 0s 173us/step - loss: 0.6932 - acc: 0.4936 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 190/200
2800/2800 [==============================] - 0s 169us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6933 - val_acc: 0.4817
Epoch 191/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6933 - val_acc: 0.4817
Epoch 192/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6932 - val_acc: 0.4817
Epoch 193/200
2800/2800 [==============================] - 0s 171us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6933 - val_acc: 0.4817
Epoch 194/200
2800/2800 [==

2800/2800 [==============================] - 0s 173us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6934 - val_acc: 0.4817
Epoch 199/200
2800/2800 [==============================] - 1s 179us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6934 - val_acc: 0.4817
Epoch 200/200
2800/2800 [==============================] - 0s 178us/step - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6933 - val_acc: 0.4817


In [44]:
#getting the accuracy on validation set on last epoch

loss, acc = final_model.evaluate(validx, validy, verbose = 0)
print("Accuracy: " + str(acc * 100))

Accuracy: 48.16666666666667
